In [2]:
#import thư viện 
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib 
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#đọc dữ liệu
df_booking = pd.read_csv('hotel_bookings.csv')
df_booking.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


Giải thích các trường dữ liệu:
- hotel: loại khách sạn (city hotel/resort hotel)
- is_canceled: có hủy booking không (1 có, 0 không)
- lead_time: khoảng thời gian từ lúc đặt phòng cho tới ngày nhận phòng 
- arrival_date_year: thời gian đến nhận phòng - năm 
- arrival_date_month: thời gian đến nhận phòng - tháng
- arrival_date_week_number: thời gian đến nhận phòng - tuần
- arrival_date_day_of_month: thời gian đến nhận phòng - ngày
- stays_in_weekend_nights: số đêm cuối tuần ở lại (chỉ tính T7/CN)
- stay_in_week_nights: số đêm ở lại (tính từ T2 - T6)
- adults: số người lớn 
- children: số trẻ em 
- babies: số trẻ sơ sinh 
- meal: bữa ăn (BB: bed and breakfast, HB: half board, FB: full board, SC: Self_care, undefined)
- country: quốc tịch 
- market_segment: phân khúc thị trường (online/offline TA/TO: online/offline travel agency/travel operate, groups, direct, corporate, complementary, aviation, undefined)
- distribution_channel: kênh phân phối (TA/TO, direct, corporate, GDS - global distribution system, undefined)
- is_repeated_guest: có phải khách quay lại không (1 có, 0 không)
- previous_cancellations:số lần hủy trước đó  
- previous_bookings_not_canceled: số lần không hủy trước đó
- reserved_room_type: loại phòng lúc đặt
- assigned_room_type: loại phòng lúc nhận 
- booking_changes: có thay đổi đặt chỗ không (1 có, 0 không)
- deposit_type: loại deposit (no deposit: không cần đặt, non refund: đặt cọc không hoàn lại, refundable: đặt cọc có hoàn lại)
- agent: đại lý 
- company: công ty 
- days_in_waiting_list: số ngày trong danh sách đợi 
- customer_type: loại khách (Transient: tạm thời, Transient-Party: tạm thời qua trung gian, Contract: khách hợp đồng, Group: khách đoàn)
- adr: giá phòng trung bình/ngày  
- required_car_parking_spaces: yêu cầu về không gian đỗ xe 
- total_of_special_requests: tổng số yêu cầu đặc biệt 
- reservation_status: trạng thái đặt chỗ
- reservation_status_date: ngày đặt chỗ  

In [4]:
#check dữ liệu null của các trường
df_booking.isnull().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

Check data null: 
- children: 4
- country: 488
- agent: 16340 
- company: 112593

In [5]:
#fix lỗi dữ liệu null
df_booking_cln = df_booking.fillna({'children':0,'country':'Undefined','agent':0,'company':0})

In [6]:
#thay giá trị undefined trường meal thành sc vì cùng nghĩa 
df_booking_cln['meal'].replace('Undefined','SC', inplace=True)

In [7]:
#bỏ những dòng dữ liệu mà số khách người lớn, trẻ em và sơ sinh đều bằng 0
guests_0 = list(df_booking_cln.loc[df_booking_cln["adults"] + df_booking_cln["children"] + df_booking_cln["babies"]==0].index)
df_booking_cln.drop(df_booking_cln.index[guests_0], inplace=True)

In [8]:
df_booking_cln['is_canceled'].value_counts()

0    75011
1    44199
Name: is_canceled, dtype: int64

In [9]:
import sqlite3

In [10]:
cnn = sqlite3.connect('bookings_hotel.db')
df_booking_cln.to_sql('booking', cnn)
%load_ext sql 
%sql sqlite:///bookings_hotel.db

### 1. Lượt khách đến hàng năm (city hotel và resort)? 

In [11]:
booking_by_year = %sql select arrival_date_year, hotel, count(*) as num_booking from booking group by arrival_date_year, hotel order by arrival_date_year asc, num_booking desc
#chuyển kết quả query về dạng dataframe
booking_by_year = pd.DataFrame(booking_by_year)
#đổi tên cột dataframe vừa tạo
booking_by_year = booking_by_year.rename(columns={0:'Year',1:'Hotel',2:'Number of booking'})
booking_by_year['grow_rate'] = booking_by_year['Number of booking'].pct_change()
booking_by_year

 * sqlite:///bookings_hotel.db
Done.


,Year,Hotel,Number of booking,grow_rate
0,2015,City Hotel,13658,NaN
1,2015,Resort Hotel,8309,-0.391639
2,2016,City Hotel,38059,3.580455
3,2016,Resort Hotel,18564,-0.512231
4,2017,City Hotel,27446,0.478453
5,2017,Resort Hotel,13174,-0.520003


In [63]:
booking_by_year.to_csv('booking_by_year.csv', header = True)

In [43]:
# writing to Excel
booking_by_year_excel = pd.ExcelWriter('booking_by_year.xlsx')
  
# write data to excel
booking_by_year.to_excel(booking_by_year_excel)

### 2. Tháng mấy lượng booking cao nhất? 

In [12]:
booking_by_month = %sql select arrival_date_year, arrival_date_month, count(*) as num_booking from booking group by arrival_date_year, arrival_date_month order by arrival_date_year, arrival_date_week_number
#chuyển kết quả query về dạng dataframe
booking_by_month = pd.DataFrame(booking_by_month)
#đổi tên cột dataframe vừa tạo
booking_by_month = booking_by_month.rename(columns={0:'Year',1:'Month',2:'Number of booking'})
booking_by_month['grow_rate'] = booking_by_month['Number of booking'].pct_change()
booking_by_month

 * sqlite:///bookings_hotel.db
Done.


,Year,Month,Number of booking,grow_rate
0,2015,July,2774,NaN
1,2015,August,3883,0.399784
2,2015,September,5109,0.315735
3,2015,October,4951,-0.030926
4,2015,November,2337,-0.527974
5,2015,December,2913,0.246470
6,2016,January,2248,-0.228287
7,2016,February,3884,0.727758
8,2016,March,4819,0.240731
9,2016,April,5425,0.125752


In [64]:
booking_by_month.to_csv('booking_by_month.csv', header = True)

In [44]:
# writing to Excel
booking_by_month_excel = pd.ExcelWriter('booking_by_month.xlsx')
  
# write data to excel
booking_by_month.to_excel(booking_by_month_excel)

### 3. Lượng booking bị cancel theo tháng?

In [13]:
booking_canceled_by_month = %sql select arrival_date_year, arrival_date_month, count(*) as num_booking from booking where is_canceled = 1 group by arrival_date_year, arrival_date_month order by arrival_date_year, arrival_date_week_number
#chuyển kết quả query về dạng dataframe
booking_canceled_by_month = pd.DataFrame(booking_canceled_by_month)
#đổi tên cột dataframe vừa tạo
booking_canceled_by_month = booking_canceled_by_month.rename(columns={0:'Year',1:'Month',2:'Number of booking'})
booking_canceled_by_month['grow_rate'] = booking_canceled_by_month['Number of booking'].pct_change()
booking_canceled_by_month

 * sqlite:///bookings_hotel.db
Done.


,Year,Month,Number of booking,grow_rate
0,2015,July,1259,NaN
1,2015,August,1598,0.269261
2,2015,September,2093,0.309762
3,2015,October,1732,-0.172480
4,2015,November,486,-0.719400
5,2015,December,973,1.002058
6,2016,January,557,-0.427544
7,2016,February,1335,1.396768
8,2016,March,1477,0.106367
9,2016,April,2060,0.394719


In [65]:
booking_canceled_by_month.to_csv('booking_canceled_by_month.csv', header = True)

In [45]:
# writing to Excel
booking_canceled_by_month_excel = pd.ExcelWriter('booking_canceled_by_month.xlsx')
  
# write data to excel
booking_canceled_by_month.to_excel(booking_canceled_by_month_excel)

### 4. Loại khách hàng nào hay cancel booking nhất?

In [14]:
cust_cancel_booking = %sql select customer_type, count(*) as num_booking from booking where is_canceled = 1 group by customer_type order by num_booking desc
#chuyển kết quả query về dạng dataframe
cust_cancel_booking = pd.DataFrame(cust_cancel_booking)
#đổi tên cột dataframe vừa tạo
cust_cancel_booking = cust_cancel_booking.rename(columns={0:'Customer Type',1:'Number of booking'})
cust_cancel_booking

 * sqlite:///bookings_hotel.db
Done.


,Customer Type,Number of booking
0,Transient,36494
1,Transient-Party,6385
2,Contract,1262
3,Group,58


In [66]:
cust_cancel_booking.to_csv('cust_cancel_booking.csv', header = True)

In [46]:
# writing to Excel
cust_cancel_booking_excel = pd.ExcelWriter('cust_cancel_booking.xlsx')
  
# write data to excel
cust_cancel_booking.to_excel(cust_cancel_booking_excel)

### 5. Lead time trung bình giữa số booking bị cancel và số booking không bị cancel?

In [15]:
avg_lead_time = %sql select case when is_canceled = 1 then 'Canceled' else 'Non-cancel' end as is_cancel, sum(lead_time)/count(*) as avg_lead_time from booking group by is_cancel order by avg_lead_time desc
#chuyển kết quả query về dạng dataframe
avg_lead_time = pd.DataFrame(avg_lead_time)
#đổi tên cột dataframe vừa tạo
avg_lead_time = avg_lead_time.rename(columns={0:'Is cancel',1:'Average of lead time'})
avg_lead_time

 * sqlite:///bookings_hotel.db
Done.


,Is cancel,Average of lead time
0,Canceled,144
1,Non-cancel,80


In [67]:
avg_lead_time.to_csv('avg_lead_time.csv', header = True)

In [47]:
# writing to Excel
avg_lead_time_excel = pd.ExcelWriter('avg_lead_time.xlsx')
  
# write data to excel
avg_lead_time.to_excel(avg_lead_time_excel)

### 6. Lượng booking ở cả cuối tuần trên tổng số booking thành công?

In [50]:
booking_weekend = %sql select b1.In_weekend, count(*) as num_booking from (select *, case when stays_in_weekend_nights > 0 then 'stay in weekend' else 'not stay in weekend' end as In_weekend from booking) as b1 group by b1.In_weekend order by num_booking desc
#chuyển kết quả query về dạng dataframe
booking_weekend = pd.DataFrame(booking_weekend)
#đổi tên cột dataframe vừa tạo
booking_weekend = booking_weekend.rename(columns={0:'In weekend',1:'Number of booking'})
booking_weekend

 * sqlite:///bookings_hotel.db
Done.


,In weekend,Number of booking
0,stay in weekend,67315
1,not stay in weekend,51895


In [68]:
booking_weekend.to_csv('booking_weekend.csv', header = True)

In [51]:
# writing to Excel
booking_weekend_excel = pd.ExcelWriter('booking_weekend.xlsx')
  
# write data to excel
booking_weekend.to_excel(booking_weekend_excel)

### 7. Lượng booking có trẻ em và babies trên tổng số booking thành công

In [52]:
booking_child = %sql select b1.have_child, count(*) as num_booking from (select *, case when children > 0 or babies > 0 then 'have child or babies' else 'not have child or babies' end as have_child from booking) as b1 group by b1.have_child order by num_booking desc
#chuyển kết quả query về dạng dataframe
booking_child = pd.DataFrame(booking_child)
#đổi tên cột dataframe vừa tạo
booking_child = booking_child.rename(columns={0:'Have child or babies',1:'Number of booking'})
booking_child

 * sqlite:///bookings_hotel.db
Done.


,Have child or babies,Number of booking
0,not have child or babies,109878
1,have child or babies,9332


In [69]:
booking_child.to_csv('booking_child.csv', header = True)

In [53]:
# writing to Excel
booking_child_excel = pd.ExcelWriter('booking_child.xlsx')
  
# write data to excel
booking_child.to_excel(booking_child_excel)

### 8. Nhu cầu dùng bữa của khách (loại meal nào hay được lựa chọn nhất khi booking)?

In [19]:
meal_booking = %sql select meal, count(*) as num_booking from booking group by meal order by num_booking desc
#chuyển kết quả query về dạng dataframe
meal_booking = pd.DataFrame(meal_booking)
#đổi tên cột dataframe vừa tạo
meal_booking = meal_booking.rename(columns={0:'Meal',1:'Number of booking'})
meal_booking

 * sqlite:///bookings_hotel.db
Done.


,Meal,Number of booking
0,BB,92236
1,HB,14458
2,SC,11718
3,FB,798


In [70]:
meal_booking.to_csv('meal_booking.csv', header = True)

In [54]:
# writing to Excel
meal_booking_excel = pd.ExcelWriter('meal_booking.xlsx')
  
# write data to excel
meal_booking.to_excel(meal_booking_excel)

### 9. Khách đến từ những nước nào?

In [20]:
country_booking = %sql select country, count(*) as num_booking from booking group by country order by num_booking desc 
#chuyển kết quả query về dạng dataframe
country_booking = pd.DataFrame(country_booking)
#đổi tên cột dataframe vừa tạo
country_booking = country_booking.rename(columns={0:'Country',1:'Number of booking'})
country_booking

 * sqlite:///bookings_hotel.db
Done.


,Country,Number of booking
0,PRT,48483
1,GBR,12120
2,FRA,10401
3,ESP,8560
4,DEU,7285
...,...,...
173,BFA,1
174,BDI,1
175,ATF,1
176,ASM,1


In [71]:
country_booking.to_csv('country_booking.csv', header = True)

In [55]:
# writing to Excel
country_booking_excel = pd.ExcelWriter('country_booking.xlsx')
  
# write data to excel
country_booking.to_excel(country_booking_excel)

### 10. Phân khúc nào trên thị trường nào phổ biến nhất?

In [21]:
market_segment_popular = %sql select market_segment, count(*) as num_booking from booking group by market_segment order by num_booking desc
#chuyển kết quả query về dạng dataframe
market_segment_popular = pd.DataFrame(market_segment_popular)
#đổi tên cột dataframe vừa tạo
market_segment_popular = market_segment_popular.rename(columns={0:'Market segment',1:'Number of booking'})
market_segment_popular

 * sqlite:///bookings_hotel.db
Done.


,Market segment,Number of booking
0,Online TA,56408
1,Offline TA/TO,24182
2,Groups,19791
3,Direct,12582
4,Corporate,5282
5,Complementary,728
6,Aviation,235
7,Undefined,2


In [72]:
market_segment_popular.to_csv('market_segment_popular.csv', header = True)

In [56]:
# writing to Excel
market_segment_popular_excel = pd.ExcelWriter('market_segment_popular.xlsx')
  
# write data to excel
market_segment_popular.to_excel(market_segment_popular_excel)

### 11. Kênh phân phối nào thông dụng nhất?

In [22]:
channel_popular = %sql select distribution_channel, count(*) as num_booking from booking group by distribution_channel order by Distribution_channel
#chuyển kết quả query về dạng dataframe
channel_popular = pd.DataFrame(channel_popular)
#đổi tên cột dataframe vừa tạo
channel_popular = channel_popular.rename(columns={0:'Distribution channel',1:'Number of booking'})
channel_popular

 * sqlite:///bookings_hotel.db
Done.


,Distribution channel,Number of booking
0,Corporate,6651
1,Direct,14611
2,GDS,193
3,TA/TO,97750
4,Undefined,5


In [73]:
channel_popular.to_csv('channel_popular.csv', header = True)

In [57]:
# writing to Excel
channel_popular_excel = pd.ExcelWriter('channel_popular.xlsx')
  
# write data to excel
channel_popular.to_excel(channel_popular_excel)

### 12. Kênh phân phối nào có tỷ lệ cancel booking cao nhất?

In [23]:
channel_popular_cancel = %sql select distribution_channel, count(*) as num_booking from booking where is_canceled = 1 group by distribution_channel order by distribution_channel
#chuyển kết quả query về dạng dataframe
channel_popular_cancel = pd.DataFrame(channel_popular_cancel)
#đổi tên cột dataframe vừa tạo
channel_popular_cancel = channel_popular_cancel.rename(columns={0:'Distribution channel',1:'Number of booking'})
channel_popular_cancel

 * sqlite:///bookings_hotel.db
Done.


,Distribution channel,Number of booking
0,Corporate,1467
1,Direct,2555
2,GDS,37
3,TA/TO,40136
4,Undefined,4


In [74]:
channel_popular_cancel.to_csv('channel_popular_cancel.csv', header = True)

In [24]:
channel_popular['Cancel rate'] = channel_popular_cancel['Number of booking']/channel_popular['Number of booking']*100
channel_popular

,Distribution channel,Number of booking,Cancel rate
0,Corporate,6651,22.056834
1,Direct,14611,17.486825
2,GDS,193,19.170984
3,TA/TO,97750,41.059847
4,Undefined,5,80.000000


In [58]:
# writing to Excel
channel_popular_cancel_excel = pd.ExcelWriter('channel_popular_cancel.xlsx')
  
# write data to excel
channel_popular_cancel.to_excel(channel_popular_cancel_excel)

### 13. Tỷ lệ khách quay lại trên tổng số booking?

In [26]:
return_guests = %sql select 100*(select count(*) from booking where is_repeated_guest = 1)/(select count(*) from booking)
return_guests

 * sqlite:///bookings_hotel.db
Done.


100*(select count(*) from booking where is_repeated_guest = 1)/(select count(*) from booking)
3


### 14. Loại phòng nào được đặt nhiều nhất?

In [40]:
reserved_room_type = %sql select hotel, reserved_room_type, count(*) as num_booking from booking group by hotel, reserved_room_type order by hotel, num_booking desc
#chuyển kết quả query về dạng dataframe
reserved_room_type = pd.DataFrame(reserved_room_type)
#đổi tên cột dataframe vừa tạo
reserved_room_type = reserved_room_type.rename(columns={0:'Hotel',1:'Room type',2:'Number of booking'})
reserved_room_type

 * sqlite:///bookings_hotel.db
Done.


,Hotel,Room type,Number of booking
0,City Hotel,A,62484
1,City Hotel,D,11747
2,City Hotel,F,1788
3,City Hotel,E,1537
4,City Hotel,B,1112
5,City Hotel,G,482
6,City Hotel,C,13
7,Resort Hotel,A,23389
8,Resort Hotel,D,7432
9,Resort Hotel,E,4982


In [75]:
reserved_room_type.to_csv('reserved_room_type.csv', header = True)

In [59]:
# writing to Excel
reserved_room_type_excel = pd.ExcelWriter('reserved_room_type.xlsx')
  
# write data to excel
reserved_room_type.to_excel(reserved_room_type_excel)

### 15. Giá phòng trung bình (city hotel. resort)?

In [42]:
price_avg = %sql select distinct hotel, reserved_room_type, avg(adr) as average_adr from booking group by hotel, reserved_room_type order by hotel, average_adr desc
#chuyển kết quả query về dạng dataframe
price_avg = pd.DataFrame(price_avg)
#đổi tên cột dataframe vừa tạo
price_avg = price_avg.rename(columns={0:'Hotel',1:'Room type',2:'Average of price per night'})
price_avg

 * sqlite:///bookings_hotel.db
Done.


,Hotel,Room type,Average of price per night
0,City Hotel,G,202.637842
1,City Hotel,F,189.576689
2,City Hotel,E,158.143162
3,City Hotel,D,131.679145
4,City Hotel,A,96.401704
5,City Hotel,C,92.057692
6,City Hotel,B,90.515027
7,Resort Hotel,H,188.223111
8,Resort Hotel,G,168.238658
9,Resort Hotel,C,161.356536


In [76]:
price_avg.to_csv('price_avg.csv', header = True)

In [60]:
# writing to Excel
price_avg_excel = pd.ExcelWriter('price_avg.xlsx')
  
# write data to excel
price_avg.to_excel(price_avg_excel)

### 16. Tỷ lệ lượng booking có loại phòng lúc đặt và lúc nhận khác nhau?

In [36]:
reserve_assign = %sql select arrival_date_year, hotel, count(*) as num_booking from booking where reserved_room_type != assigned_room_type group by arrival_date_year, hotel order by arrival_date_year
#chuyển kết quả query về dạng dataframe
reserve_assign = pd.DataFrame(reserve_assign)
#đổi tên cột dataframe vừa tạo
reserve_assign = reserve_assign.rename(columns={0:'Year',1:'Hotel',2:'Number of booking'})
reserve_assign['change_room_type_rate'] = reserve_assign['Number of booking']/booking_by_year['Number of booking']*100
reserve_assign

 * sqlite:///bookings_hotel.db
Done.


,Year,Hotel,Number of booking,change_room_type_rate
0,2015,City Hotel,1951,14.284668
1,2015,Resort Hotel,1863,22.421471
2,2016,City Hotel,3724,9.784808
3,2016,Resort Hotel,3851,20.744452
4,2017,City Hotel,1403,5.111856
5,2017,Resort Hotel,2004,15.211781


In [77]:
reserve_assign.to_csv('reserve_assign.csv', header = True)

In [61]:
# writing to Excel
reserve_assign_excel = pd.ExcelWriter('reserve_assign.xlsx')
  
# write data to excel
reserve_assign.to_excel(reserve_assign_excel)